# Necessary pre-work that we need to do 

- Go to the Google APIs Console.
- Create a new project.
- Click Enable Google Drive API and Google Sheet API
- Create credentials for a Web Server to access Application Data.
- Name the service account and grant it a Project Role of Editor.
- Download the JSON file to the desired location and rename it as "creds.json'"
- Open and read creds.json and there is a key which is called "client_email"
- Share the google sheet you want to update with the 'client email' 

For more details and steps, please visit there https://www.youtube.com/watch?v=cnPlKLEGR7E&t=428s 
After you follow all the steps from the video, we can start with python code 

# Import libraries

In [1]:
#If you did not install gspread, you need to install it first
#!pip install gspread oauth2client 

In [3]:
# import necessary libraries
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
import pandas as pd

# Open the creds.json file


In [5]:
with open('creds.json') as f:
    creds = json.load(f)

In [15]:
# take a look at your own creds.json
creds['client_email'] # <<< ---- please share your google sheet with this email

You can see simiary content with your own creds.json. 

# Read Data from a Spreadsheet

In [11]:
gc = gspread.service_account(filename='creds.json') # gspread – to interact with Google Spreadsheets

In [12]:
sheet = gc.open("Consumer_Behavior_Data") # or by sheet name: gc.open("TestList")
sheet_instance = sheet.get_worksheet(0) #For the first sheet, pass the index 0 and so on.

In [13]:
# get all the records of the data
records_data = sheet_instance.get_all_records()
# convert the json to dataframe
records_df = pd.DataFrame(records_data)
# view the top records
records_df.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Timestamp,Year,Month,Day,Hour,Gender,Clicked on Ad,Age Range
0,68.95,35,"$61,833.90",256.09,Cloned 5thgeneration orchestration,Wrightburgh,Tunisia,2016-03-27 0:53:11,2016,3,27,0,Female,Clicked,26-35
1,80.23,31,"$68,441.85",193.77,Monitored national standardization,West Jodi,Nauru,2016-04-04 1:39:02,2016,4,4,1,Male,Clicked,26-35
2,69.47,26,"$59,785.94",236.50,Organic bottom-line service-desk,Davidton,San Marino,2016-03-13 20:35:42,2016,3,13,20,Female,Clicked,26-35
3,74.15,29,"$54,806.18",245.89,Triple-buffered reciprocal time-frame,West Terrifurt,Italy,2016-01-10 2:31:19,2016,1,10,2,Male,Clicked,26-35
4,68.37,35,"$73,889.99",225.58,Robust logistical utilization,South Manuel,Iceland,2016-06-03 3:36:18,2016,6,3,3,Female,Clicked,26-35


# Update Data in Google Sheets


In [73]:
#Gender
Gender=[]
for i in records_df['Gender']:
    if i=="Female":
        i=0
        Gender.append(i)
    else:
        i=1
        Gender.append(i)
# Clicked on Ad
Clicked=[]
for i in records_df['Clicked on Ad']:
    if i=="Clicked":
        i=1
        Clicked.append(i)
    else:
        i=0
        Clicked.append(i)

In [74]:
records_df["Gender"]=Gender
records_df["Clicked on Ad"]=Clicked
records_df.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Country,Timestamp,Year,Month,Day,Hour,Gender,Clicked on Ad,Age Range
0,68.95,35,"$61,833.90",256.09,Cloned 5thgeneration orchestration,Wrightburgh,Tunisia,2016-03-27 0:53:11,2016,3,27,0,0,1,26-35
1,80.23,31,"$68,441.85",193.77,Monitored national standardization,West Jodi,Nauru,2016-04-04 1:39:02,2016,4,4,1,1,1,26-35
2,69.47,26,"$59,785.94",236.50,Organic bottom-line service-desk,Davidton,San Marino,2016-03-13 20:35:42,2016,3,13,20,0,1,26-35
3,74.15,29,"$54,806.18",245.89,Triple-buffered reciprocal time-frame,West Terrifurt,Italy,2016-01-10 2:31:19,2016,1,10,2,1,1,26-35
4,68.37,35,"$73,889.99",225.58,Robust logistical utilization,South Manuel,Iceland,2016-06-03 3:36:18,2016,6,3,3,0,1,26-35


In [75]:
# group by gender
gender_groupby=pd.DataFrame(records_df.groupby('Month')['Clicked on Ad'].value_counts())
gender_groupby.columns=['Counts']
gender_groupby=gender_groupby.reset_index()
gender_groupby

,Month,Clicked on Ad,Counts
0,1,1,78
1,1,0,69
2,2,0,83
3,2,1,77
4,3,1,82
5,3,0,74
6,4,0,74
7,4,1,73
8,5,0,79
9,5,1,68


# Update Data in Google Sheets

In [79]:
gender_groupby.shape

(14, 3)

In [82]:
# add a sheet with 20 rows and 2 columns
sheet.add_worksheet(rows=14,cols=3,title='updated_df')

# get the instance of the second sheet
new_sheet = sheet.get_worksheet(3)

In [17]:
# Update values to the sheet
new_sheet.insert_rows(gender_groupby.values.tolist())

In [16]:
# Add headers for your updated data 
header = list(gender_groupby.columns)
index = 1
new_sheet.insert_row(header, index)